In [5]:
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import networkx as nx
import gym 
from tqdm import tqdm 
from collections import deque
rng = np.random.RandomState(2025)

In [12]:
class Node:
    def __init__(self, name, parent=None, mean=0, var=1):
        self.name = name
        self.mean = mean  
        self.var = var  
        self.children = []
        self.scores_children = np.array([])
        self.nb_children = 0
        self.parent = parent
        self.level = parent.level + 1 if parent else 0

    def get_child_nodes(self):
        if not nodes:
            nodes = []
        for child in self.children:
            nodes.append((child.name, child.level, child.nb_children))
            nodes.extend(child.get_child_nodes())
        return nodes

    def get_reward(self):
        return self.mean + np.sqrt(self.var) * np.random.normal()



class Tree:
    def __init__(self):
        self.levels = [[]]
        self.graph = {'root': None,}
        self.max_level = 0

    def create_node(self, name, parent=None, mean=0, var=1):
        return Node(name, parent, mean, var)

    def insert(self, parent_node, name, mean, var):
        if parent_node is None:
            node = self.create_node(name, mean=mean, var=var)
            if node.level == 0:
                self.root = node
                self.graph['root'] = node
            return node

        node = self.create_node(name, parent_node, mean, var)
        self.graph[name] = node
        parent_node.children.append(node)
        parent_node.nb_children = len(parent_node.children)
        parent_node.scores_children = np.full(parent_node.nb_children, 1.0 / parent_node.nb_children)
        self.max_level = max(self.max_level, node.level)
        return node, parent_node

    def get_parent_nodes(self, node):
        nodes = [node]
        def _recursive_parent_nodes(node, nodes):
            if node.parent:
                nodes.append(node.parent)
                _recursive_parent_nodes(node.parent, nodes)
            return nodes
        nodes = _recursive_parent_nodes(node, nodes)
        nodes.reverse()
        return nodes

    def get_all_nodes(self):
        nodes = [self.root]
        def _get_nodes(node):
            for child in node.children:
                nodes.append(child)
                _get_nodes(child)
        _get_nodes(self.root)
        nodes = sorted(nodes, key=lambda node: node.level)
        nodes_names = [node.name for node in nodes]
        return nodes

    def get_all_leaves(self):
        leaves = []
        def get_leaves(node):
            if len(node.children) == 0:
                leaves.append(node)
            if node is not None:
                for child in node.children:
                    get_leaves(child)
        get_leaves(self.root)
        return leaves

    def get_reward_leaf(self, leaf):
        parents_leaf = self.get_parent_nodes(leaf)
        reward = 0 
        for node in parents_leaf:
            reward += node.get_reward()
        return reward
    
    def get_reward_leaves(self):
        leaves = self.get_all_leaves()
        data = []
        for leaf in leaves:
            data.append([leaf.name, self.get_reward_leaf(leaf)])
        return data


    def iterative_dfs(self, node_key=None):
        if not node_key:
            node_key = 'root'
        visited = []
        stack = deque()
        stack.append(node_key)
        while stack:
            node_key = stack.pop()
            if node_key not in visited:
                visited.append(node_key)
                unvisited = [n.name for n in self.graph[node_key].children if n.name not in visited]
                stack.extend(unvisited)
        return visited

    def iterative_bfs(self, start=None):
        if not start:
            start = 'root'
        visited = []
        queue = deque()
        queue.append(start)
        while queue:
            node_key = queue.popleft()
            if node_key not in visited:
                visited.append(node_key)
                unvisited = [n.name for n in self.graph[node_key].children if n.name not in visited]
                queue.extend(unvisited)
        return visited

    def find_max_sum_path(self, root, max_result=-np.infty, max_path=[]):
        if root is None:
            return 0, max_result
        max_sums = [0]
        max_paths = [[]]
        for child in root.children:
            max_sum, idx_max_path = self.find_max_sum_path(child, max_result, max_path)
            max_sums.append(max_sum)
            max_paths.append(idx_max_path)
        sums = root.get_reward() + np.array(max_sums)
        idx = np.argmax(sums)
        max_result = sums[idx]
        max_path = max_paths[idx]
        max_path.append(idx)
        return max_result, max_path

    def find_best_arm_path(self):
        max_mean, path = self.find_max_sum_path(self.root)
        path = np.array(path[1:])
        path = list(path - 1)[::-1]
        return max_mean, path

In [ ]:
tree = Tree()
root = tree.insert(parent_node=None, name="Utilisateur", mean=0, var=1)

# First Layer
vehicule, _ = tree.insert(parent_node=root, name="vehicule", mean=0.8, var=2)
mob_verte, _ = tree.insert(parent_node=root, name="Mobilités Vertes", mean=2, var=2)

# Second Layer
taxi, _ = tree.insert(parent_node=vehicule, name="taxi", mean=0.1, var=2)
uber, _ = tree.insert(parent_node=vehicule, name="uber", mean=0.2, var=2)
bus, _ = tree.insert(parent_node=vehicule, name="bus", mean=0.05, var=2)
tram, _ = tree.insert(parent_node=vehicule, name="tram", mean=0.3, var=2)
covoiturage, _ = tree.insert(parent_node=vehicule, name="covoiturage", mean=0.2, var=2)
avion, _ = tree.insert(parent_node=vehicule, name="avion", mean=0.1, var=2)
train, _ = tree.insert(parent_node=vehicule, name="train", mean=0.4, var=2)
rer, _ = tree.insert(parent_node=vehicule, name="RER", mean=0.3, var=2)
metro, _ = tree.insert(parent_node=vehicule, name="métro", mean=0.1, var=2)

velo, _ = tree.insert(parent_node=mob_verte, name="vélo", mean=1.5, var=2)
marche, _ = tree.insert(parent_node=mob_verte, name="marche", mean=1.0, var=2)
cap, _ = tree.insert(parent_node=mob_verte, name="course à pied", mean=2.5, var=2)


print("\n--- All the nodes in the Tree ---")
all_nodes = tree.get_all_nodes()
for node in all_nodes:
    print(f"Name: {node.name}, Drawing reward: {node.get_reward():.4f}, Level: {node.level}")

print("\n--- Leaves of the Tree ---")
data = tree.get_all_leaves()
for elements in data:
    name = elements[0]
    reward = elements[1]
    print(f"Name: {name}, Drawing reward: {reward:.4f}")

print("\n--- Path with the maximum reward ---")
max_sum, best_path = tree.find_best_arm_path()
print(f"Max sum : {max_sum:.4f}")
print(f"Path : {best_path}")


--- All the nodes in the Tree ---
Name: Utilisateur, Drawing reward: 1.0192, Level: 0
Name: vehicule, Drawing reward: 0.6502, Level: 1
Name: Mobilités Vertes, Drawing reward: 0.7313, Level: 1
Name: taxi, Drawing reward: 1.7363, Level: 2
Name: uber, Drawing reward: 1.5874, Level: 2
Name: bus, Drawing reward: -0.7455, Level: 2
Name: tram, Drawing reward: 1.4165, Level: 2
Name: covoiturage, Drawing reward: -0.3954, Level: 2
Name: avion, Drawing reward: -2.2843, Level: 2
Name: train, Drawing reward: 0.3470, Level: 2
Name: RER, Drawing reward: 0.6346, Level: 2
Name: métro, Drawing reward: -0.1620, Level: 2
Name: vélo, Drawing reward: 0.9901, Level: 2
Name: marche, Drawing reward: 0.4771, Level: 2
Name: course à pied, Drawing reward: 2.2285, Level: 2

--- Leaves of the Tree ---


AttributeError: 'Node' object has no attribute 'get_reward_leaf'

In [ ]:
def play(environment, agent, Nmc, T):
    """
    Simulate multiple trajectories of agent-environment interaction and record regret.

    This function simulates the interaction between a single agent and an environment 
    over Nmc independent trajectories, each lasting T rounds. It records the pseudo-regret 
    at each time step by comparing the received reward to the best possible reward.

    Parameters
    ----------
    environment : object
        The environment in which the agent operates. 

    agent : object
        The agent being evaluated. 

    Nmc : int
        Number of Monte Carlo trajectories to simulate.

    T : int
        Time horizon (number of steps per trajectory).

    Returns
    -------
    agent_id : str
        The name or identifier of the agent (via `agent.name()`).

    data : np.ndarray
        A NumPy array of shape (Nmc, T), where each entry contains the pseudo-regret 
        (best possible reward minus received reward) at time t in trajectory n.

    Notes
    -----
    The pseudo-regret at each time step is computed using the current mean rewards
    of the environment arms, which assumes those are fixed and known for evaluation.
    """
    # Initialize the regret matrix
    data = np.zeros((Nmc, T))

    # Loop over each independent trajectory
    for n in range(Nmc):
        environment.reset()  # Reset environment if needed (i.i.d. or not)
        agent.reset()        # Reset internal state of the agent

        for t in range(T):
            # Get available actions and let agent choose one
            action_set = environment.get_action_set()
            action = agent.get_action(action_set)

            # Get reward for the chosen action and send it back to the agent
            reward = environment.get_reward(action)
            agent.receive_reward(action, reward)

            # Compute pseudo-regret: best mean - received reward
            means = environment.get_means()
            best_reward = np.max(means)
            data[n, t] = best_reward - reward

    return agent.name(), data


In [ ]:
def experiment(environment, agents, Nmc, T):
    """
    Run multiple simulations of each agent interacting with the environment and record regret.

    This function evaluates a list of agents by having them interact with a given environment
    over multiple Monte Carlo simulations, each lasting for a fixed time horizon.
    For each agent, it computes and stores the regret over time for each trajectory.

    Parameters
    ----------
    environment : object
        The environment in which the agents operate. 
    
    agents : list
        A list of agent instances. Each agent must implement `reset()`, `get_action()`,
        `receive_reward(action, reward)`, and `name()` methods.
    
    Nmc : int
        Number of Monte Carlo simulations to run for each agent.
    
    T : int
        Time horizon (number of rounds) for each simulation.

    Returns
    -------
    all_data : dict
        A dictionary mapping each agent's name (str) to a NumPy array of shape (Nmc, T),
        containing the regret at each time step for each Monte Carlo run.
    """
    all_data = {}

    for agent in agents:
        # Run Nmc simulations for the current agent and compute regrets
        agent_id, regrets = play(environment, agent, Nmc, T)

        # Store the results in the dictionary under the agent's name
        all_data[agent_id] = regrets

    return all_data

In [ ]:
class StochasticEnvironment(gym.Env):
    """
    The Environment class represents a decision-making environment with a tree structure.
    The tree is generated dynamically and rewards are assigned to each node based on its level and value. 
    The environment allows the computation of rewards based on nodes and paths, 
    and finds the best strategy for decision-making.

    Attributes
    ----------
    env_rng : numpy.random.RandomState
        A random state generator used for sampling rewards.
    
    nb_layers : int 
        Number of layers for the Tree. 
        Example 3 

    nb_nodes_per_layer : list
        Number of nodes per layer for the Tree. 
        Example [1, 2, 5]

    mus : list
        Mean of the distribution for each layer. 
        Example [0, [1, 4], [[0.1, 0.3], [0.2, 0.1, 0.1]]]
    
    """ 
    def __init__(self, nb_nodes_per_layer, mus):
        self.env_rng = np.random.RandomState(2025)  # Use a fixed seed for reproducibility
        self.nb_layers = len(nb_nodes_per_layer) 
        self.nb_nodes_per_layer = nb_nodes_per_layer
        self.mus = mus
        self.sigma = 1

    def reset(self):
        self.tree = Tree()
        self.generate_tree()
        return [0]
    
    def _initialize_tree(self):
        """
        Initializes the tree by creating the root node.
        """
        self.root = None
        self.tree = Tree()
        self.root = self.tree.insert(self.root, ('root', self.mus[0])) 

    def generate_tree(self):
        self._initialize_tree()

        def build_layer(parents, mus_layer, layer_idx):
            current_layer_nodes = []

            if len(parents) != len(mus_layer):
                raise ValueError(f"Layer {layer_idx}: expected {len(parents)} parents, got {len(mus_layer)} mus sublists")
            
            for parent_idx, (parent_node, child_mus_list) in enumerate(zip(parents, mus_layer)):
                if not isinstance(child_mus_list, list):
                    raise ValueError(f"Layer {layer_idx}: mus[{layer_idx}][{parent_idx}] should be a list of floats")

                for child_idx, mu in enumerate(child_mus_list):
                    value = self.env_rng.normal(mu, self.sigma)
                    node_data = (f"Node_{layer_idx}_{parent_idx}_{child_idx}", value)
                    node, _ = self.tree.insert(parent_node, node_data)
                    current_layer_nodes.append(node)
            return current_layer_nodes

        if not isinstance(self.mus[0], (int, float)):
            raise ValueError("mus[0] must be a scalar for root node.")
        self.root.data = ("root", self.env_rng.normal(self.mus[0], self.sigma))

        parents = [self.root]

        for layer_idx in range(1, len(self.mus)):
            mus_layer = self.mus[layer_idx]

            if layer_idx == 1:
                current_layer_nodes = []
                for idx, mu in enumerate(mus_layer):
                    value = self.env_rng.normal(mu, self.sigma)
                    node_data = (f"Node_{layer_idx}_{idx}", value)
                    node, _ = self.tree.insert(self.root, node_data)
                    current_layer_nodes.append(node)
                parents = current_layer_nodes
            else:
                parents = build_layer(parents, mus_layer, layer_idx)

        return self.tree
    
    def get_action_set(self):
        return list(np.arange(len(self.tree.get_all_nodes())))
    
    def get_reward(self, action):
        """Obtain the reward given an action."""
        # The action is a path, with indexes corresponding to the nodes 
        node = self.tree.graph.get(action)
        if node:
            total_reward = 0
            path_node = node
            while path_node:
                total_reward += path_node.get_reward()
                path_node = path_node.parent
            return total_reward
        return 0
    
    def get_best_strategy_reward(self):
        all_rewards = [node.mean for node in self.tree.get_all_nodes()]
        return np.max(all_rewards)

    def get_reward_by_node(self, node):
        return node.get_reward()
    
    def step(self, action):
        reward = self.get_reward(action)
        done = True  
        return [0], reward, done, {}
    
    def render(self, mode='human', close=False):
        pass

    
    def visualize_tree(self):
        G = nx.DiGraph()

        def add_edges(node, parent_name=None):
            if node is not None:
                G.add_node(node.name, level=node.level) 
                if parent_name:
                    G.add_edge(parent_name, node.name)  
                for child in node.children:  
                    add_edges(child, node.name)

        add_edges(self.tree.root)
        

        color_map = plt.cm.get_cmap("viridis", self.tree.max_level + 1)
        levels = [G.nodes[node]['level'] for node in G.nodes]
        norm = plt.Normalize(vmin=min(levels), vmax=max(levels))
        node_colors = [color_map(norm(G.nodes[node]['level'])) for node in G.nodes]
        pos = nx.spring_layout(G, seed=2025)  
        labels = nx.get_node_attributes(G, 'label')
        nx.draw(G, pos, with_labels=True, node_size=3000, node_color=node_colors, font_size=10, font_weight='bold')
        sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
        sm.set_array([])
        level_labels = [f"Couche {i}" for i in range(self.tree.max_level + 1)]
        cbar = plt.colorbar(sm, shrink=2, aspect=2)
        cbar.set_ticks(np.linspace(0, 1, self.tree.max_level + 1))  
        cbar.set_ticklabels(level_labels)  
        
        plt.show()